In [1]:
!pip install -r requirements.txt


ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'

[notice] A new release of pip is available: 24.0 -> 26.0.1
[notice] To update, run: C:\Users\PRACHITI\AppData\Local\Programs\Python\Python311\python.exe -m pip install --upgrade pip


In [2]:
import os

def print_structure(path="data"):
    for root, dirs, files in os.walk(path):
        level = root.replace(path, "").count(os.sep)
        indent = " " * 2 * level
        print(f"{indent}{os.path.basename(root)}/")
        for f in files:
            print(f"{indent}  {f}")

print_structure("data")


ImportError: cannot import name 'pipeline' from 'src' (unknown location)